In [ ]:
import time
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import warnings
warnings.filterwarnings('ignore')

In [ ]:
es_remote = Elasticsearch(['10.127.122.198'], 
                   port = 60500,
                   verify_certs = False,
                   use_ssl = True)
es_remote.ping()

In [ ]:
es_remote.info()

In [ ]:
es_local = Elasticsearch()
def bulk_baseline_data(index, mlist):
    for m in mlist:
    # use a `yield` generator so that the data
    # isn't loaded into memory
        yield {
            "_index": index,
            "_source": m,
        }
        
es_doc_list = []
while True:
    thread_pool_data = es_remote.cat.thread_pool(format = 'json')
    # df = pd.DataFrame(thread_pool_data)
    # print(df)
    search_pool_info = thread_pool_data[13]
    write_pool_info = thread_pool_data[19]

#     print(search_pool_info)
#     print(write_pool_info)
    
    
    jvm_data = es_remote.nodes.stats(metric = ["jvm"])
    node = list(jvm_data["nodes"].keys())[0]
    jvm_mem_info = jvm_data["nodes"][node]["jvm"]["mem"]
    print(jvm_mem_info)
    
    es_doc = {
        "timestamp": time.time(),
        "search_active": search_pool_info['active'],
        "search_queue": search_pool_info['queue'],
        "search_rejected": search_pool_info['rejected'],
        "write_active": write_pool_info['active'],
        "write_queue": write_pool_info['queue'],
        "write_rejected": write_pool_info['rejected'],
        "heap_used_percentage": jvm_mem_info['heap_used_percent']
    }
    es_doc_list.append(es_doc.copy())
    time.sleep(1)
    if len(es_doc_list) >= 60:
        response = helpers.bulk(es_local, bulk_baseline_data('health_monitoring', es_doc_list))
        es_doc_list.clear()
        print("Pushed {} records to elastic db".format(response[0]))
        

In [ ]:
es.xpack.client.monitoring.transport.hosts

In [ ]:
es.transport.hosts

In [ ]:
from datetime import datetime
import time

# now = datetime.now()
now = time.time()
print(now)

new_now = now.replace(minute = (now.minute // 15) * 15, second = 0, microsecond = 0)
print(new_now)

In [ ]:
datetime.fromtimestamp(1627920326).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
import time

# now = datetime.now()
now = time.localtime(time.time())
print(now)

In [ ]:
now = time.time()
t_now = time.localtime(time.time())
print(now, t_now)
t_ref = (t_now.tm_year, t_now.tm_mon, t_now.tm_mday-t_now.tm_wday, 0, 0, 0, 0, 0, 0)
ref = time.mktime(t_ref)
print(ref, t_ref)

time_diff = now - ref
sid = int(time_diff//30)
print(sid)

In [ ]:
import numpy as np
a = np.array([1,2,3,4])
print(a.shape)

In [ ]:
import getopt
argv = ['h', '-c', 28, '-v', 25]
opts, args = getopt.getopt(argv, "hc:v:")
for opt, arg in opts:
    print(opt, arg)


In [ ]:
queryQ = {"query" : {"bool": {"must": []}}}

queryQ

In [ ]:
import pandas as pd
from datetime import datetime
import time


outlier_data = [{'timestamp': 1630662896845, 'rdAvgEctOutScr': 2.8397917247301083, 'rdMaxEctOutScr': 0, 'rdAbtFailTimeoutOutScr': 0.5, 'wrAvgEctOutScr': 0, 'wrMaxEctOutScr': 0, 'wrAbtFailTimeoutOutScr': 0}, 
                {'timestamp': 1630661876845, 'rdAvgEctOutScr': 1.4600929990130465, 'rdMaxEctOutScr': 0, 'rdAbtFailTimeoutOutScr': 0, 'wrAvgEctOutScr': 3.184716406159366, 'wrMaxEctOutScr': 0, 'wrAbtFailTimeoutOutScr': 0}, 
                {'timestamp': 1630663076845, 'rdAvgEctOutScr': 2.1499423618715774, 'rdMaxEctOutScr': 0, 'rdAbtFailTimeoutOutScr': 2.0, 'wrAvgEctOutScr': 0, 'wrMaxEctOutScr': 0, 'wrAbtFailTimeoutOutScr': 0}, 
                {'timestamp': 1630662986845, 'rdAvgEctOutScr': 3.8745657690179045, 'rdMaxEctOutScr': 0, 'rdAbtFailTimeoutOutScr': 0.75, 'wrAvgEctOutScr': 0, 'wrMaxEctOutScr': 0, 'wrAbtFailTimeoutOutScr': 0}]
    
gte = '2021-09-03T09:31:20.000Z'
lte = '2021-09-03T10:01:20.000Z'

gte_time = datetime.strptime(gte, "%Y-%m-%dT%H:%M:%S.%fZ")
gte_epoch_time = int((gte_time - datetime(1970, 1, 1)).total_seconds() * 1000)
print(gte_time, gte_epoch_time)

lte_time = datetime.strptime(lte, "%Y-%m-%dT%H:%M:%S.%fZ")
lte_epoch_time = int((lte_time - datetime(1970, 1, 1)).total_seconds() * 1000)
print(lte_time, lte_epoch_time)

df = pd.DataFrame(outlier_data)

ts_min = df.timestamp.min()
ts_max = df.timestamp.max()

placeholder_df = pd.DataFrame()
while gte_epoch_time < ts_min - 30000:
    tmp_df = pd.DataFrame([ts_min - 30000], columns=['timestamp'])
    placeholder_df = pd.concat([placeholder_df, tmp_df])
    ts_min -= 30000
    
while ts_max + 30000 < lte_epoch_time:
    tmp_df = pd.DataFrame([ts_max + 30000], columns=['timestamp'])
    placeholder_df = pd.concat([placeholder_df, tmp_df])
    ts_max += 30000
    
count = 1
for i in range(df.shape[0]-1):
    while df.timestamp[i+1] > df.timestamp[i] + count*30000:
        tmp_df = pd.DataFrame([df.timestamp[i] + count*30000], columns=['timestamp'])
        placeholder_df = pd.concat([placeholder_df, tmp_df])
        count += 1
        
# tmp_df1 = pd.DataFrame([1630661480000], columns=['timestamp'])
# tmp_df2 = pd.DataFrame([1630663280000], columns=['timestamp'])
# df = pd.concat([df, tmp_df1, tmp_df2])


# r = pd.date_range(start=df.timestamp.min(), end=df.timestamp.max(), freq='30S')
# df2 = df.set_index('timestamp').reindex(r).fillna(0).rename_axis('timestamp').reset_index()
df = pd.concat([df, placeholder_df])
df = df.sort_values('timestamp', axis = 0)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.strftime("%Y-%m-%dT%H:%M:%SZ")
df = df.fillna(0)
print(df.shape)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

In [ ]:
type(df.shape[0])

In [ ]:
(1630662896845 - 1630661876845)/(60 * 1000)


In [ ]:
import pandas as pd
import time
from datetime import datetime


outlier_data = [{'timestamp': 1630662896845, 'rdAvgEctOutScr': 2.8397917247301083, 'rdMaxEctOutScr': 0, 'rdAbtFailTimeoutOutScr': 0.5, 'wrAvgEctOutScr': 0, 'wrMaxEctOutScr': 0, 'wrAbtFailTimeoutOutScr': 0}, 
                {'timestamp': 1630661876845, 'rdAvgEctOutScr': 1.4600929990130465, 'rdMaxEctOutScr': 0, 'rdAbtFailTimeoutOutScr': 0, 'wrAvgEctOutScr': 3.184716406159366, 'wrMaxEctOutScr': 0, 'wrAbtFailTimeoutOutScr': 0}, 
                {'timestamp': 1630663076845, 'rdAvgEctOutScr': 2.1499423618715774, 'rdMaxEctOutScr': 0, 'rdAbtFailTimeoutOutScr': 2.0, 'wrAvgEctOutScr': 0, 'wrMaxEctOutScr': 0, 'wrAbtFailTimeoutOutScr': 0}, 
                {'timestamp': 1630662986845, 'rdAvgEctOutScr': 3.8745657690179045, 'rdMaxEctOutScr': 0, 'rdAbtFailTimeoutOutScr': 0.75, 'wrAvgEctOutScr': 0, 'wrMaxEctOutScr': 0, 'wrAbtFailTimeoutOutScr': 0}]


    
gte = '2021-09-03T09:31:20Z'
lte = '2021-09-03T10:01:20Z'
        
df = pd.DataFrame(outlier_data)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.strftime("%Y-%m-%dT%H:%M:%SZ")

# tmp_df1 = pd.DataFrame([gte], columns=['timestamp'])
# tmp_df2 = pd.DataFrame([lte], columns=['timestamp'])
# df = pd.concat([df, tmp_df1, tmp_df2])

print(df.timestamp.min())
r = pd.date_range(start=df.timestamp.min(), end=df.timestamp.max(), freq='30S')
print(r)
df = df.set_index('timestamp').reindex(r).rename_axis('timestamp').reset_index()


df = df.sort_values('timestamp', axis = 0)
df.head(100)


In [ ]:
a = "umang"
print(a[:-1])

In [ ]:
umang_dict = dict()

print(umang_dict)

In [ ]:
import json
import pprint

with open('sample_tier_input.json') as f: 
    tier_data = json.load(f)
    pprint.pprint(tier_data)

In [ ]:
t = int(input())
for i in range(t):
    ans = False
    x = input()
    a,b,c,d,e = map(int, x.split())
    res = [a,b,c]
    res.sort(reverse = True)
    i = 0
    while i < len(res):
        if res[i] <= e:
            res.remove(res[i])
            break
        i+=1
        
    if len(res) == 3:
        print("NO")
        continue
        
    if sum(res) <= d:
        print("YES")
    else:
        print("NO")
        
        
    

In [1]:
import pandas as pd
flow_docs_list = [{'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.392, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.393, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.506, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 12.506, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.724, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.724, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.392, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.393, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.332, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.333, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.17, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.171, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.418, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.419, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.328, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.329, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.106, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.106, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.418, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.419, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.02, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.02, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.798, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.799, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.336, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.337, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.588, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.589, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.356, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.357, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.842, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 13.842, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.106, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.106, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.26, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 13.26, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.666, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 13.666, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.064, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.065, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.608, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.608, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.068, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.068, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.224, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.224, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.89, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 12.891, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.978, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 15.979, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.06, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.06, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.564, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.565, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.232, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.233, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.796, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.797, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.868, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.868, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.956, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.956, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.978, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 15.979, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.848, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.849, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.17, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.171, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.492, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.493, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.798, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.799, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.39, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.391, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.232, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.233, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.382, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 15.382, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.382, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.382, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.492, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.493, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.514, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 15.515, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.66, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.66, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.068, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.068, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.89, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 12.891, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.938, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 15.939, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.336, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.337, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.064, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.065, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.224, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.224, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.986, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 12.986, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.884, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.885, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.884, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.885, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.246, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.246, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.06, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.06, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.512, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.513, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.332, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.333, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.512, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.513, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.592, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.592, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.612, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.613, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.66, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.66, 'rdAvgInterIOGap': 1}]
flow_docs_list = flow_docs_list*1000

%timeit pd.DataFrame(flow_docs_list)
%timeit pd.DataFrame.from_dict(flow_docs_list)
%timeit pd.DataFrame.from_records(flow_docs_list)


111 ms ± 4.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
110 ms ± 5.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
110 ms ± 4.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
import numpy as np
from numpy import mean, absolute
import pandas as pd
import statistics
from scipy import stats
from statsmodels import robust
from astropy.stats import median_absolute_deviation
from numpy.random import randn

rand = np.random.default_rng(12345)
a = rand.standard_normal(30)
print(len(a), a)

print("******************")

%timeit sum(a)/len(a)
%timeit np.mean(a)

print("******************")

%timeit min(a)
%timeit np.min(a)

print("******************")

%timeit np.median(a)
%timeit statistics.median(a)

print("******************")

%timeit pd.DataFrame(a).mad()
print(pd.DataFrame(a).mad())
%timeit stats.median_abs_deviation(a)
%timeit median_absolute_deviation(a)
%timeit robust.mad(a, c=1)
%timeit mean(absolute(a - mean(a)))
print(mean(absolute(a - mean(a))))

30 [-1.42382504  1.26372846 -0.87066174 -0.25917323 -0.07534331 -0.74088465
 -1.3677927   0.6488928   0.36105811 -1.95286306  2.34740965  0.96849691
 -0.75938718  0.90219827 -0.46695317 -0.06068952  0.78884434 -1.25666813
  0.57585751  1.39897899  1.32229806 -0.29969852  0.90291934 -1.62158273
 -0.15818926  0.44948393 -1.34360107 -0.08168759  1.72473993  2.61815943]
******************
4.72 µs ± 76.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.56 µs ± 52.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
******************
2.93 µs ± 53.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.95 µs ± 69.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
******************
28.2 µs ± 447 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
5.31 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
******************
827 µs ± 20.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
0    0.974922
dtype: float64
100 µs 

In [7]:
import numpy as np
from numpy import mean, absolute
import pandas as pd
import statistics
from scipy import stats
from statsmodels import robust
from astropy.stats import median_absolute_deviation

import random
# res = [random.randrange(1, 60, 1) for i in range(30)]

# print("******************")

# %timeit sum(a)/len(a)
# %timeit np.mean(a)

# print("******************")

# %timeit min(a)
# %timeit np.min(a)

# print("******************")

# %timeit np.median(a)
# %timeit statistics.median(a)

# print("******************")

# %timeit pd.DataFrame(a).mad()
# print(pd.DataFrame(a).mad())

# %timeit stats.median_abs_deviation(a)
# print(stats.median_abs_deviation(a))

# %timeit median_absolute_deviation(a)
# %timeit robust.mad(a, c=1)
# %timeit mean(absolute(a - mean(a)))

%timeit statistics.median(absolute(a - statistics.median(a)))
print("a = {} median(a) = {}".format(a, np.median(a)))
print(statistics.median(absolute(a - statistics.median(a))))

12.5 µs ± 414 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
a = [-1.42382504  1.26372846 -0.87066174 -0.25917323 -0.07534331 -0.74088465
 -1.3677927   0.6488928   0.36105811 -1.95286306  2.34740965  0.96849691
 -0.75938718  0.90219827 -0.46695317 -0.06068952  0.78884434 -1.25666813
  0.57585751  1.39897899  1.32229806 -0.29969852  0.90291934 -1.62158273
 -0.15818926  0.44948393 -1.34360107 -0.08168759  1.72473993  2.61815943] median(a) = -0.06801641287377447
0.9135377222395007


In [8]:
type(a)

numpy.ndarray

In [17]:
x = [1,2,3,4]
print(type(x), x)

x = np.array(x)
print(type(x), x)
print(x-1)

<class 'list'> [1, 2, 3, 4]


TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [10]:
a

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331,
       -0.74088465, -1.3677927 ,  0.6488928 ,  0.36105811, -1.95286306,
        2.34740965,  0.96849691, -0.75938718,  0.90219827, -0.46695317,
       -0.06068952,  0.78884434, -1.25666813,  0.57585751,  1.39897899,
        1.32229806, -0.29969852,  0.90291934, -1.62158273, -0.15818926,
        0.44948393, -1.34360107, -0.08168759,  1.72473993,  2.61815943])

In [9]:
list(a) - 1

TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [32]:
import modin.pandas as pd
flow_docs_list = [{'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.392, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.393, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.506, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 12.506, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.724, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.724, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.392, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.393, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.332, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.333, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.17, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.171, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.418, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.419, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.328, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.329, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.106, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.106, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.418, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.419, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.02, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.02, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.798, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.799, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.336, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.337, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.588, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.589, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.356, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.357, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.842, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 13.842, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.106, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.106, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.26, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 13.26, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.666, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 13.666, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.064, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.065, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.608, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.608, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.068, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.068, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.224, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.224, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.89, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 12.891, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.978, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 15.979, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.06, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.06, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.564, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.565, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.232, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.233, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.796, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.797, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.868, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.868, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.956, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.956, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.978, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 15.979, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.848, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.849, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.17, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.171, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.492, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.493, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.798, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.799, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.39, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.391, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.232, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.233, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.382, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 15.382, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.382, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.382, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.492, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.493, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.514, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 15.515, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.66, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.66, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.068, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.068, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.89, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 12.891, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.938, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 15.939, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.336, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.337, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.064, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.065, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.224, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.224, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.986, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 12.986, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.884, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.885, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.884, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.885, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.246, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.246, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.06, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.06, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.512, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.513, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.332, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.333, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.512, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.513, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.592, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.592, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.612, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.613, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.66, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.66, 'rdAvgInterIOGap': 1}]

# %timeit pd.DataFrame(flow_docs_list)
# %timeit pd.DataFrame.from_dict(flow_docs_list)
# %timeit pd.DataFrame.from_records(flow_docs_list)


In [33]:
df = pd.DataFrame(flow_docs_list)
df.head()

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()



,rdAvgIOPs,tier,wrAvgIOPs,rdAvgSizePerIO,baselineStatus,wrAvgECTPerIO,wrAvgSizePerIO,flowId,wrAvgInterIOGap,timestamp,rdAvgECTPerIO,rdAvgInterIOGap
0,33.333332,0,16.666666,4096,init,19.392,8192,8000,1.5,1642396289733,14.393,1
1,33.333332,0,16.666666,4096,init,17.506,8192,20000,1.5,1642396289733,12.506,1
2,33.333332,0,16.666666,4096,init,19.724,8192,28000,1.5,1642396289733,14.724,1
3,33.333332,0,16.666666,4096,init,19.392,8192,36000,1.5,1642396289733,14.393,1
4,33.333332,0,16.666666,4096,init,19.332,8192,24000,1.5,1642396289733,14.333,1


In [34]:
# df.set_index("flowId", inplace=True)
df.head()

,rdAvgIOPs,tier,wrAvgIOPs,rdAvgSizePerIO,baselineStatus,wrAvgECTPerIO,wrAvgSizePerIO,flowId,wrAvgInterIOGap,timestamp,rdAvgECTPerIO,rdAvgInterIOGap
0,33.333332,0,16.666666,4096,init,19.392,8192,8000,1.5,1642396289733,14.393,1
1,33.333332,0,16.666666,4096,init,17.506,8192,20000,1.5,1642396289733,12.506,1
2,33.333332,0,16.666666,4096,init,19.724,8192,28000,1.5,1642396289733,14.724,1
3,33.333332,0,16.666666,4096,init,19.392,8192,36000,1.5,1642396289733,14.393,1
4,33.333332,0,16.666666,4096,init,19.332,8192,24000,1.5,1642396289733,14.333,1


In [35]:
metrics_list = ['timestamp', 'rdAvgIOPs', 'wrAvgIOPs', 'rdAvgSizePerIO', 'wrAvgSizePerIO', 'rdAvgInterIOGap', 'wrAvgInterIOGap', 'rdAvgECTPerIO', 'wrAvgECTPerIO']

agg_data_df = df.groupby('flowId').agg({y: lambda x:list(x) for y in metrics_list})
agg_data_df.head()


,timestamp,rdAvgIOPs,wrAvgIOPs,rdAvgSizePerIO,wrAvgSizePerIO,rdAvgInterIOGap,wrAvgInterIOGap,rdAvgECTPerIO,wrAvgECTPerIO
flowId,,,,,,,,,
4000,"[1642396199733, 1642396259733, 1642396229733, ...","[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.419, 12.891, 15.979, 13.337, 13.885, 15.513]","[19.418, 17.89, 20.978, 18.336, 18.884, 20.512]"
8000,"[1642396289733, 1642396199733, 1642396229733, ...","[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.393, 15.106, 14.799, 13.493, 14.233, 14.66]","[19.392, 20.106, 19.798, 18.492, 19.232, 19.66]"
12000,"[1642396319733, 1642396199733, 1642396169733, ...","[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.065, 14.068, 14.224, 14.06, 15.171, 14.333]","[19.064, 19.068, 19.224, 19.06, 20.17, 19.332]"
16000,"[1642396199733, 1642396169733, 1642396229733, ...","[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.419, 13.337, 15.979, 12.891, 13.885, 15.513]","[19.418, 18.336, 20.978, 17.89, 18.884, 20.512]"
20000,"[1642396289733, 1642396199733, 1642396169733, ...","[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[12.506, 15.02, 13.357, 13.26, 13.666, 14.246]","[17.506, 20.02, 18.356, 18.26, 18.666, 19.246]"


In [36]:
agg_data_df.index

Int64Index([4000, 8000, 12000, 16000, 20000, 24000, 28000, 32000, 36000,
            40000],
           dtype='int64', name='flowId')

In [38]:
data = agg_data_df.loc[4000]
data["timestamp"]

[1642396199733,
 1642396259733,
 1642396229733,
 1642396169733,
 1642396319733,
 1642396289733]

In [19]:
agg_data_df.loc[4000]

rdAvgIOPs          [33.333332, 33.333332, 33.333332, 33.333332, 3...
wrAvgIOPs          [16.666666, 16.666666, 16.666666, 16.666666, 1...
rdAvgSizePerIO     [4096, 4096, 4096, 4096, 4096, 4096, 4096, 409...
wrAvgSizePerIO     [8192, 8192, 8192, 8192, 8192, 8192, 8192, 819...
rdAvgInterIOGap    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
wrAvgInterIOGap    [1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, ...
rdAvgECTPerIO      [14.419, 12.891, 15.979, 13.337, 13.885, 15.51...
wrAvgECTPerIO      [19.418, 17.89, 20.978, 18.336, 18.884, 20.512...
Name: 4000, dtype: object

In [11]:
df.loc[8000]

,rdAvgIOPs,tier,wrAvgIOPs,rdAvgSizePerIO,baselineStatus,wrAvgECTPerIO,wrAvgSizePerIO,wrAvgInterIOGap,timestamp,rdAvgECTPerIO,rdAvgInterIOGap
flowId,,,,,,,,,,,
8000,33.333332,0,16.666666,4096,init,19.392,8192,1.5,1642396289733,14.393,1
8000,33.333332,0,16.666666,4096,init,20.106,8192,1.5,1642396199733,15.106,1
8000,33.333332,0,16.666666,4096,init,19.798,8192,1.5,1642396229733,14.799,1
8000,33.333332,0,16.666666,4096,init,18.492,8192,1.5,1642396319733,13.493,1
8000,33.333332,0,16.666666,4096,init,19.232,8192,1.5,1642396259733,14.233,1
...,...,...,...,...,...,...,...,...,...,...,...
8000,33.333332,0,16.666666,4096,init,20.106,8192,1.5,1642396199733,15.106,1
8000,33.333332,0,16.666666,4096,init,19.798,8192,1.5,1642396229733,14.799,1
8000,33.333332,0,16.666666,4096,init,18.492,8192,1.5,1642396319733,13.493,1


In [13]:
import time
init = time.time()
ans = []
flow_docs_list = [{'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.392, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.393, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.506, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 12.506, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.724, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.724, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.392, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.393, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.332, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.333, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.17, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.171, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.418, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.419, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.328, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.329, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.106, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.106, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.418, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.419, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.02, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.02, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.798, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.799, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.336, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.337, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.588, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.589, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.356, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.357, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.842, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 13.842, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.106, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.106, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.26, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 13.26, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.666, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 13.666, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.064, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.065, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.608, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.608, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.068, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.068, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.224, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.224, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.89, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 12.891, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.978, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 15.979, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.06, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.06, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.564, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.565, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.232, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.233, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.796, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.797, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.868, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.868, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.956, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.956, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.978, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 15.979, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.848, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.849, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.17, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.171, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.492, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.493, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.798, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.799, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.39, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.391, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.232, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.233, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.382, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 15.382, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.382, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.382, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.492, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.493, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.514, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 15.515, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.66, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.66, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.068, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.068, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.89, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 12.891, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.938, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 15.939, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.336, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.337, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.064, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.065, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.224, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.224, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.986, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 12.986, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.884, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.885, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.884, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.885, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.246, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.246, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.06, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.06, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.512, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.513, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.332, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.333, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.512, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.513, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.592, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.592, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.612, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.613, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.66, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.66, 'rdAvgInterIOGap': 1}]
df = pd.DataFrame(flow_docs_list)
df.set_index('flowId', inplace=True)

for flowid in df.index:
    gg_2 = df.loc[flowid]
    ans.append(gg_2["rdAvgECTPerIO"])
    ans.append(gg_2["wrAvgECTPerIO"])
    
end = time.time()

print("Time taken = ", end-init)
    

Time taken =  1.2785131931304932


In [7]:
import time
init = time.time()
import pandas as pd
import numpy as np
from scipy import stats

flow_docs_list = [{'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.392, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.393, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.506, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 12.506, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.724, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.724, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.392, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.393, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.332, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.333, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.17, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.171, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.418, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.419, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.328, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.329, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.106, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.106, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.418, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.419, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.02, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.02, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.798, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.799, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.336, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.337, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.588, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.589, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.356, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.357, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.842, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 13.842, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.106, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 15.106, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.26, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 13.26, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.666, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 13.666, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.064, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.065, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.608, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.608, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.068, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.068, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.224, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.224, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.89, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 12.891, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.978, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 15.979, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.06, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.06, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.564, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.565, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.232, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.233, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.796, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.797, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.868, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.868, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.956, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.956, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.978, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 15.979, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.848, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.849, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.17, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 15.171, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.492, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.493, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.798, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.799, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.39, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.391, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.232, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 14.233, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.382, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 15.382, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.382, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.382, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.492, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.493, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.514, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 15.515, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.66, 'wrAvgSizePerIO': 8192, 'flowId': 36000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.66, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.068, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396199733, 'rdAvgECTPerIO': 14.068, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.89, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396259733, 'rdAvgECTPerIO': 12.891, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.938, 'wrAvgSizePerIO': 8192, 'flowId': 28000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 15.939, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.336, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 13.337, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.064, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.065, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.224, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.224, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 17.986, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 12.986, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.884, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.885, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 18.884, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 13.885, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.246, 'wrAvgSizePerIO': 8192, 'flowId': 20000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396319733, 'rdAvgECTPerIO': 14.246, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.06, 'wrAvgSizePerIO': 8192, 'flowId': 24000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396229733, 'rdAvgECTPerIO': 14.06, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.512, 'wrAvgSizePerIO': 8192, 'flowId': 4000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.513, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.332, 'wrAvgSizePerIO': 8192, 'flowId': 12000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.333, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.512, 'wrAvgSizePerIO': 8192, 'flowId': 16000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.513, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.592, 'wrAvgSizePerIO': 8192, 'flowId': 40000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 14.592, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 20.612, 'wrAvgSizePerIO': 8192, 'flowId': 32000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396289733, 'rdAvgECTPerIO': 15.613, 'rdAvgInterIOGap': 1}, {'rdAvgIOPs': 33.333332, 'tier': 0, 'wrAvgIOPs': 16.666666, 'rdAvgSizePerIO': 4096, 'baselineStatus': 'init', 'wrAvgECTPerIO': 19.66, 'wrAvgSizePerIO': 8192, 'flowId': 8000, 'wrAvgInterIOGap': 1.5, 'timestamp': 1642396169733, 'rdAvgECTPerIO': 14.66, 'rdAvgInterIOGap': 1}]
df = pd.DataFrame(flow_docs_list)
df.set_index('flowId', inplace=True)

func_mean = lambda x : sum(x)/len(x)
func_mean.__name__ = 'mean'

func_min = lambda x : min(x)
func_min.__name__ = 'min'

func_list = lambda x : list(x)
# func_list.__name__ = 'list'


# metrics_list = ["timestamp", "rdAvgIOPs", "wrAvgIOPs", "rdAvgSizePerIO", "wrAvgSizePerIO", "rdAvgECTPerIO", "wrAvgECTPerIO", "rdAvgInterIOGap", "wrAvgInterIOGap"]
metrics_list = ['rdAvgIOPs', 'wrAvgIOPs', 'rdAvgSizePerIO', 'wrAvgSizePerIO', 'rdAvgInterIOGap', 'wrAvgInterIOGap', 'rdAvgECTPerIO', 'wrAvgECTPerIO']

grouped_df = df.groupby('flowId').agg({y: lambda x:list(x) for y in metrics_list})
# grouped_df = df.groupby('flowId').agg({'rdAvgIOPs': func_list, 'timestamp': [func_min, func_list], 'rdAvgECTPerIO': [np.mean, np.median], 'wrAvgECTPerIO': [np.mean, np.median, stats.median_absolute_deviation]})

end = time.time()

print("Time taken = ", end-init)
print(grouped_df.loc[4000]['rdAvgIOPs'])
grouped_df


Time taken =  0.011216163635253906
[33.333332, 33.333332, 33.333332, 33.333332, 33.333332, 33.333332]


,rdAvgIOPs,wrAvgIOPs,rdAvgSizePerIO,wrAvgSizePerIO,rdAvgInterIOGap,wrAvgInterIOGap,rdAvgECTPerIO,wrAvgECTPerIO
flowId,,,,,,,,
4000,"[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.419, 12.891, 15.979, 13.337, 13.885, 15.513]","[19.418, 17.89, 20.978, 18.336, 18.884, 20.512]"
8000,"[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.393, 15.106, 14.799, 13.493, 14.233, 14.66]","[19.392, 20.106, 19.798, 18.492, 19.232, 19.66]"
12000,"[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.065, 14.068, 14.224, 14.06, 15.171, 14.333]","[19.064, 19.068, 19.224, 19.06, 20.17, 19.332]"
16000,"[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.419, 13.337, 15.979, 12.891, 13.885, 15.513]","[19.418, 18.336, 20.978, 17.89, 18.884, 20.512]"
20000,"[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[12.506, 15.02, 13.357, 13.26, 13.666, 14.246]","[17.506, 20.02, 18.356, 18.26, 18.666, 19.246]"
24000,"[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.333, 15.171, 14.068, 14.065, 14.224, 14.06]","[19.332, 20.17, 19.068, 19.064, 19.224, 19.06]"
28000,"[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.724, 14.329, 14.608, 14.849, 14.391, 15.939]","[19.724, 19.328, 19.608, 19.848, 19.39, 20.938]"
32000,"[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.589, 14.797, 14.868, 15.382, 12.986, 15.613]","[19.588, 19.796, 19.868, 20.382, 17.986, 20.612]"
36000,"[33.333332, 33.333332, 33.333332, 33.333332, 3...","[16.666666, 16.666666, 16.666666, 16.666666, 1...","[4096, 4096, 4096, 4096, 4096, 4096]","[8192, 8192, 8192, 8192, 8192, 8192]","[1, 1, 1, 1, 1, 1]","[1.5, 1.5, 1.5, 1.5, 1.5, 1.5]","[14.393, 15.106, 14.233, 14.799, 13.493, 14.66]","[19.392, 20.106, 19.232, 19.798, 18.492, 19.66]"


In [98]:
a = range(100)
print(a)
print(func_mean(a))

range(0, 100)
49.5


In [73]:
data = grouped_df.loc[4000]
data["rdAvgIOPs"]


mean    33.333332
Name: 4000, dtype: float64

df.columns

In [ ]:
from collections import Counter
counter = Counter(df.status)
counter
print(sum(counter.values()))

In [ ]:
a = [1,2,3,4,5]
b = [6,7,8]

for i,j in zip(a, b):
    print(i,j)

In [ ]:
import time
time.time()

In [73]:
from tqdm import tqdm

a = range(10000000000)
count = 1
for item in tqdm(a):
    count += 1

  0%|          | 18629611/10000000000 [00:05<45:15, 3676123.19it/s]


KeyboardInterrupt: 